# Kaggle Dogs VS Cats

## Basic Lenet-5

In [25]:
import random
import os

import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    
from tensorflow.keras import layers, callbacks, metrics, optimizers, models
import keras_tuner as kt
import numpy as np
import skimage.io as io

In [26]:
class DogCatSequence(tf.keras.utils.Sequence):
    
    def __init__(self, files_folder, files_list, batch_size):
        if batch_size > len(files_list):
            raise ValueError('Batch size is bigger than length of files list')
            
        self._files_list = files_list
        self._batch_size = batch_size
        self._files_folder = files_folder

    def __getitem__(self, index):
        end = min((index + 1) * self._batch_size, len(self._files_list) - 1)
        batch_files_list = self._files_list[index * self._batch_size: end]

        batch_x = np.array([io.imread(os.path.join(self._files_folder, filename)) for filename in batch_files_list])
        batch_y = np.array([float(filename.startswith('dog')) for filename in batch_files_list])

        return batch_x, batch_y

    def __len__(self):
        return int(np.ceil(len(self._files_list) / self._batch_size))


def f_score(ytrue, ypred, threshold=0.5, epsilon=10e-7):
    # casting ytrue and ypred as float dtype
    ytrue = tf.cast(ytrue, tf.float32)
    ypred = tf.cast(ypred, tf.float32)

    # setting values of ypred greater than the set threshold to 1 while those lesser to 0
    ypred = tf.cast(tf.greater_equal(ypred, tf.constant(threshold)), tf.float32)

    tp = tf.reduce_sum(ytrue*ypred) # calculating true positives
    predicted_positive = tf.reduce_sum(ypred) # calculating predicted positives
    actual_positive = tf.reduce_sum(ytrue) # calculating actual positives
    
    precision = tp/(predicted_positive+epsilon) # calculating precision
    recall = tp/(actual_positive+epsilon) # calculating recall
    
    # calculating fbeta
    fb = 2 * precision*recall / (precision + recall + epsilon)

    return fb


def print_errors(model, train_seq, test_seq):
    train_f_score = model.evaluate(train_seq)[1]
    test_f_score = model.evaluate(test_seq)[1]

    print('train error = ', 1 - train_f_score)
    print('test error = ', 1 - test_f_score)
    
    
def train_model(model, trainin_sequence, validation_sequence):
    model_check_point = callbacks.ModelCheckpoint(filepath='dogs_vs_cats_lenet5_weights_{epoch}.h5',
                                              save_weights_only=True,
                                              save_best_only=True,
                                              monitor='val_loss',
                                              mode='min',
                                              verbose=True)

    early_stopping = callbacks.EarlyStopping(monitor='val_loss',
                                            mode='min',
                                            min_delta=10e-3,
                                            verbose=True,
                                            patience=30,
                                            restore_best_weights=True)
    
    lr_plateau = tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.2,
        patience=5,
        verbose=True,
        mode="min",
        min_delta=0.0001,
        cooldown=0,
        min_lr=0)



    train_epochs = 100

    train_callbacks = [model_check_point, early_stopping, lr_plateau]

    model.fit(trainin_sequence, epochs=train_epochs, validation_data=validation_sequence, callbacks=train_callbacks)
    
    return model

def tune_model(model_func, train_sequence, validation_sequence):
    hp = kt.HyperParameters()

    tuner = kt.Hyperband(
        model_func,
        objective="val_loss",
        max_epochs=30,
        hyperband_iterations=3,
        factor=5,
        directory='tuner',
        project_name=str(model_func),
    )

    train_epochs = 10
    tuner.search(train_sequence, epochs=train_epochs, validation_data=validation_sequence)
    
    return tuner

def get_dataset_sequences(images_folder, batch_size):
    images_list = sorted(os.listdir(images_folder))
    random.seed(42)
    random.shuffle(images_list)

    train_part = 0.9
    val_part = 0.05
    test_part = 0.05

    get_last_index = lambda part: int(part * len(images_list))

    train_images_list = images_list[: get_last_index(train_part)]
    val_images_list = images_list[get_last_index(train_part): get_last_index(train_part + val_part)]
    test_images_list = images_list[get_last_index(train_part + val_part):
                                       get_last_index(train_part + val_part + test_part)]
    train_seq = DogCatSequence(images_folder, train_images_list, batch_size)
    val_seq = DogCatSequence(images_folder, val_images_list, batch_size)
    test_seq = DogCatSequence(images_folder, test_images_list, batch_size)
    
    return train_seq, val_seq, test_seq

In [55]:
train_seq, val_seq, test_seq = get_dataset_sequences('train-64')

## Basic Lenet-5

In [10]:
def lenet5_v1(input_shape):
    inputs = layers.Input(shape=input_shape)
    
    nn = layers.Rescaling(1.0 / 255.0)(inputs)
    nn = layers.Conv2D(filters=6, kernel_size=5, strides=(1, 1), activation='relu')(inputs)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)
    nn = layers.Conv2D(filters=16, kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)
    nn = layers.Flatten()(nn)
    nn = layers.Dense(400, activation='relu')(nn)
    nn = layers.Dense(120, activation='relu')(nn)
    nn = layers.Dense(84, activation='relu')(nn)
    outputs = layers.Dense(1, activation='sigmoid')(nn)
    return models.Model(inputs, outputs)

In [11]:
model = lenet5_v1(train_seq[0][0].shape[1:])

train_metrics = [f_score,
                 metrics.Precision(thresholds=0.5),
                 metrics.Recall(thresholds=0.5)]

train_optimizer = optimizers.Adam(learning_rate=0.001)

train_loss = 'binary_crossentropy'

model.compile(loss=train_loss,
              optimizer=train_optimizer,
              metrics=train_metrics)

In [12]:
train_model(model)

Epoch 1/100
351/352 [============================>.] - ETA: 0s - loss: 1.4011 - f_score: 0.5428 - precision_1: 0.5430 - recall_1: 0.6137
Epoch 1: val_loss improved from inf to 0.68042, saving model to dogs_vs_cats_lenet5_weights_1.h5
352/352 [==============================] - 11s 29ms/step - loss: 1.3990 - f_score: 0.5430 - precision_1: 0.5430 - recall_1: 0.6139 - val_loss: 0.6804 - val_f_score: 0.5652 - val_precision_1: 0.5822 - val_recall_1: 0.5584
Epoch 2/100
351/352 [============================>.] - ETA: 0s - loss: 0.6473 - f_score: 0.6199 - precision_1: 0.6176 - recall_1: 0.6442
Epoch 2: val_loss improved from 0.68042 to 0.65341, saving model to dogs_vs_cats_lenet5_weights_2.h5
352/352 [==============================] - 10s 28ms/step - loss: 0.6474 - f_score: 0.6202 - precision_1: 0.6180 - recall_1: 0.6447 - val_loss: 0.6534 - val_f_score: 0.6624 - val_precision_1: 0.5955 - val_recall_1: 0.7476
Epoch 3/100
352/352 [==============================] - ETA: 0s - loss: 0.6033 - f_scor

In [13]:
print_errors(model, train_seq, test_seq)

20/20 [==============================] - 1s 27ms/step - loss: 0.6495 - f_score: 0.6705 - precision_1: 0.6068 - recall_1: 0.7595
train error =  0.2862144112586975
test error =  0.329542338848114


## Basic Lenet-5 with tuning

In [14]:
def lenet5_v2(hp):
    
    inputs = layers.Input(shape=(64, 64, 3))
    nn = layers.Rescaling(1.0 / 255.0)(inputs)

    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_1', min_value=1, max_value=10, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(inputs)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)

    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_2', min_value=10, max_value=20, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)
    
    nn = layers.Flatten()(nn)

    nn = layers.Dense(hp.Int(name='units_dense_1', min_value=300, max_value=500, step=1), activation='relu')(nn)
    nn = layers.Dense(hp.Int(name='units_dense_2', min_value=50, max_value=120, step=1), activation='relu')(nn)
    nn = layers.Dense(hp.Int(name='units_dense_3', min_value=60, max_value=100, step=1), activation='relu')(nn)
    outputs = layers.Dense(1, activation='sigmoid')(nn)

    model = models.Model(inputs, outputs)
    
    train_metrics = [f_score,
                     metrics.Precision(thresholds=0.5),
                     metrics.Recall(thresholds=0.5)]

    train_optimizer = optimizers.Adam(
        learning_rate=hp.Float(name='learning_rate', min_value=10e-6, max_value=10e-4, sampling='log'))
    train_loss = 'binary_crossentropy'

    model.compile(loss=train_loss, optimizer=train_optimizer, metrics=train_metrics)
    return model

In [21]:
tuner = tune_model(lenet5_v2)

Trial 30 Complete [00h 02m 32s]
val_loss: 0.5534921884536743

Best val_loss So Far: 0.515401303768158
Total elapsed time: 00h 22m 10s
INFO:tensorflow:Oracle triggered exit


In [22]:
lenet5_v2_best_model = tuner.get_best_models()[0]
lenet5_v2_best_model = train_model(lenet5_v2_best_model)

Epoch 1/100
351/352 [============================>.] - ETA: 0s - loss: 0.4684 - f_score: 0.7750 - precision: 0.7768 - recall: 0.7819
Epoch 1: val_loss improved from inf to 0.50380, saving model to dogs_vs_cats_lenet5_weights_1.h5
352/352 [==============================] - 11s 29ms/step - loss: 0.4685 - f_score: 0.7751 - precision: 0.7766 - recall: 0.7822 - val_loss: 0.5038 - val_f_score: 0.7344 - val_precision: 0.8192 - val_recall: 0.6719
Epoch 2/100
352/352 [==============================] - ETA: 0s - loss: 0.4109 - f_score: 0.8078 - precision: 0.8117 - recall: 0.8113
Epoch 2: val_loss did not improve from 0.50380
352/352 [==============================] - 10s 28ms/step - loss: 0.4109 - f_score: 0.8078 - precision: 0.8117 - recall: 0.8113 - val_loss: 0.5213 - val_f_score: 0.7787 - val_precision: 0.7263 - val_recall: 0.8454
Epoch 3/100
350/352 [============================>.] - ETA: 0s - loss: 0.3421 - f_score: 0.8461 - precision: 0.8458 - recall: 0.8524
Epoch 3: val_loss did not impro

In [23]:
print_errors(lenet5_v2_best_model, train_seq, test_seq)

20/20 [==============================] - 1s 26ms/step - loss: 0.5472 - f_score: 0.6901 - precision: 0.7789 - recall: 0.6297
train error =  0.1928519606590271
test error =  0.3098623752593994


## Lenet-5 with additional conv layer

In [24]:
def lenet5_v3(hp):
    
    inputs = layers.Input(shape=(64, 64, 3))
    nn = layers.Rescaling(1.0 / 255.0)(inputs)

    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_1', min_value=1, max_value=20, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(inputs)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)

    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_2', min_value=10, max_value=30, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)

    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_3', min_value=20, max_value=40, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)

    
    nn = layers.Flatten()(nn)

    nn = layers.Dense(hp.Int(name='units_dense_1', min_value=300, max_value=500, step=1), activation='relu')(nn)
    nn = layers.Dense(hp.Int(name='units_dense_2', min_value=50, max_value=120, step=1), activation='relu')(nn)
    nn = layers.Dense(hp.Int(name='units_dense_3', min_value=60, max_value=100, step=1), activation='relu')(nn)
    outputs = layers.Dense(1, activation='sigmoid')(nn)

    model = models.Model(inputs, outputs)
    
    train_metrics = [f_score,
                     metrics.Precision(thresholds=0.5),
                     metrics.Recall(thresholds=0.5)]

    train_optimizer = optimizers.Adam(
        learning_rate=hp.Float(name='learning_rate', min_value=10e-6, max_value=10e-4, sampling='log'))
    train_loss = 'binary_crossentropy'

    model.compile(loss=train_loss, optimizer=train_optimizer, metrics=train_metrics)
    return model

In [25]:
tuner = tune_model(lenet5_v3)

Trial 30 Complete [00h 02m 34s]
val_loss: 0.5273486375808716

Best val_loss So Far: 0.4456341564655304
Total elapsed time: 00h 29m 04s
INFO:tensorflow:Oracle triggered exit


In [26]:
lenet5_v3_best_model = tuner.get_best_models()[0]
lenet5_v3_best_model = train_model(lenet5_v3_best_model)

Epoch 1/100
351/352 [============================>.] - ETA: 0s - loss: 0.3684 - f_score: 0.8283 - precision: 0.8333 - recall: 0.8299
Epoch 1: val_loss improved from inf to 0.43387, saving model to dogs_vs_cats_lenet5_weights_1.h5
352/352 [==============================] - 11s 30ms/step - loss: 0.3684 - f_score: 0.8284 - precision: 0.8335 - recall: 0.8301 - val_loss: 0.4339 - val_f_score: 0.8081 - val_precision: 0.7725 - val_recall: 0.8517
Epoch 2/100
351/352 [============================>.] - ETA: 0s - loss: 0.3341 - f_score: 0.8490 - precision: 0.8499 - recall: 0.8529
Epoch 2: val_loss did not improve from 0.43387
352/352 [==============================] - 10s 29ms/step - loss: 0.3345 - f_score: 0.8488 - precision: 0.8502 - recall: 0.8523 - val_loss: 0.4832 - val_f_score: 0.7683 - val_precision: 0.8349 - val_recall: 0.7177
Epoch 3/100
351/352 [============================>.] - ETA: 0s - loss: 0.3072 - f_score: 0.8629 - precision: 0.8626 - recall: 0.8686
Epoch 3: val_loss did not impro

In [27]:
print_errors(lenet5_v3_best_model, train_seq, test_seq)

20/20 [==============================] - 1s 27ms/step - loss: 0.4642 - f_score: 0.7864 - precision: 0.7400 - recall: 0.8465
train error =  0.14125597476959229
test error =  0.21361839771270752


## Lenet5 with two additional conv layers and 128x128 images

In [4]:
train_seq, val_seq, test_seq = get_dataset_sequences('train-128')

(64, 128, 128, 3)


In [5]:
def lenet5_v4(hp):
    
    inputs = layers.Input(shape=(128, 128, 3))
    nn = layers.Rescaling(1.0 / 255.0)(inputs)

    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_1', min_value=1, max_value=20, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(inputs)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)

    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_2', min_value=20, max_value=40, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)

    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_3', min_value=40, max_value=60, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)
    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_4', min_value=60, max_value=80, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)

    
    nn = layers.Flatten()(nn)

    nn = layers.Dense(hp.Int(name='units_dense_1', min_value=300, max_value=500, step=1), activation='relu')(nn)
    nn = layers.Dense(hp.Int(name='units_dense_2', min_value=80, max_value=120, step=1), activation='relu')(nn)
    nn = layers.Dense(hp.Int(name='units_dense_3', min_value=40, max_value=80, step=1), activation='relu')(nn)
    outputs = layers.Dense(1, activation='sigmoid')(nn)

    model = models.Model(inputs, outputs)
    
    train_metrics = [f_score,
                     metrics.Precision(thresholds=0.5),
                     metrics.Recall(thresholds=0.5)]

    train_optimizer = optimizers.Adam(
        learning_rate=hp.Float(name='learning_rate', min_value=10e-6, max_value=10e-4, sampling='log'))
    train_loss = 'binary_crossentropy'

    model.compile(loss=train_loss, optimizer=train_optimizer, metrics=train_metrics)
    return model

In [6]:
tuner = tune_model(lenet5_v4, train_seq, val_seq)

Trial 30 Complete [00h 07m 11s]
val_loss: 0.38811683654785156

Best val_loss So Far: 0.35724955797195435
Total elapsed time: 01h 09m 22s
INFO:tensorflow:Oracle triggered exit


In [7]:
lenet5_v4_best_model = tuner.get_best_models()[0]
lenet5_v4_best_model = train_model(lenet5_v4_best_model)

Epoch 1/100
352/352 [==============================] - ETA: 0s - loss: 0.2491 - f_score: 0.8883 - precision: 0.8867 - recall: 0.8942
Epoch 1: val_loss improved from inf to 0.33639, saving model to dogs_vs_cats_lenet5_weights_1.h5
352/352 [==============================] - 28s 78ms/step - loss: 0.2491 - f_score: 0.8883 - precision: 0.8867 - recall: 0.8942 - val_loss: 0.3364 - val_f_score: 0.8575 - val_precision: 0.8460 - val_recall: 0.8754
Epoch 2/100
352/352 [==============================] - ETA: 0s - loss: 0.2141 - f_score: 0.9077 - precision: 0.9054 - recall: 0.9130
Epoch 2: val_loss did not improve from 0.33639
352/352 [==============================] - 27s 77ms/step - loss: 0.2141 - f_score: 0.9077 - precision: 0.9054 - recall: 0.9130 - val_loss: 0.4146 - val_f_score: 0.8201 - val_precision: 0.9074 - val_recall: 0.7571
Epoch 3/100
352/352 [==============================] - ETA: 0s - loss: 0.1729 - f_score: 0.9270 - precision: 0.9266 - recall: 0.9304
Epoch 3: val_loss did not impro

In [8]:
print_errors(lenet5_v4_best_model, train_seq, test_seq)

20/20 [==============================] - 1s 34ms/step - loss: 0.3997 - f_score: 0.8284 - precision: 0.8120 - recall: 0.8544
train error =  0.08293211460113525
test error =  0.17155903577804565


## Lenet5 with three additional conv layers and 256x256 images

In [3]:
train_seq, val_seq, test_seq = get_dataset_sequences('train-256', 16)

In [4]:
def lenet5_v5(hp):
    
    inputs = layers.Input(shape=(256, 256, 3))
    nn = layers.Rescaling(1.0 / 255.0)(inputs)

    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_1', min_value=1, max_value=20, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(inputs)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)

    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_2', min_value=20, max_value=40, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)

    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_3', min_value=40, max_value=60, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)
    
    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_4', min_value=60, max_value=80, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)
    
    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_5', min_value=80, max_value=100, step=1),
                        kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)

    
    nn = layers.Flatten()(nn)

    nn = layers.Dense(hp.Int(name='units_dense_1', min_value=300, max_value=500, step=1), activation='relu')(nn)
    nn = layers.Dense(hp.Int(name='units_dense_2', min_value=80, max_value=120, step=1), activation='relu')(nn)
    nn = layers.Dense(hp.Int(name='units_dense_3', min_value=40, max_value=80, step=1), activation='relu')(nn)
    outputs = layers.Dense(1, activation='sigmoid')(nn)

    model = models.Model(inputs, outputs)
    
    train_metrics = [f_score,
                     metrics.Precision(thresholds=0.5),
                     metrics.Recall(thresholds=0.5)]

    train_optimizer = optimizers.Adam(
        learning_rate=hp.Float(name='learning_rate', min_value=10e-6, max_value=10e-4, sampling='log'))
    train_loss = 'binary_crossentropy'

    model.compile(loss=train_loss, optimizer=train_optimizer, metrics=train_metrics)
    return model

In [5]:
tuner = tune_model(lenet5_v5, train_seq, val_seq)

Trial 30 Complete [00h 31m 26s]
val_loss: 0.22479461133480072

Best val_loss So Far: 0.2145802527666092
Total elapsed time: 04h 46m 47s
INFO:tensorflow:Oracle triggered exit


In [6]:
lenet5_v5_best_model = tuner.get_best_models()[0]
lenet5_v5_best_model = train_model(lenet5_v5_best_model)

Epoch 1/100
704/704 [==============================] - ETA: 0s - loss: 0.1142 - f_score: 0.9531 - precision: 0.9490 - recall: 0.9600
Epoch 1: val_loss improved from inf to 0.22811, saving model to dogs_vs_cats_lenet5_weights_1.h5
704/704 [==============================] - 91s 128ms/step - loss: 0.1142 - f_score: 0.9531 - precision: 0.9490 - recall: 0.9600 - val_loss: 0.2281 - val_f_score: 0.9051 - val_precision: 0.9121 - val_recall: 0.9006
Epoch 2/100
704/704 [==============================] - ETA: 0s - loss: 0.0986 - f_score: 0.9600 - precision: 0.9568 - recall: 0.9657
Epoch 2: val_loss did not improve from 0.22811
704/704 [==============================] - 90s 128ms/step - loss: 0.0986 - f_score: 0.9600 - precision: 0.9568 - recall: 0.9657 - val_loss: 0.2978 - val_f_score: 0.8880 - val_precision: 0.9324 - val_recall: 0.8486
Epoch 3/100
704/704 [==============================] - ETA: 0s - loss: 0.0851 - f_score: 0.9668 - precision: 0.9655 - recall: 0.9696
Epoch 3: val_loss did not imp

In [8]:
print_errors(tuner.get_best_models()[0], train_seq, test_seq)

40/40 [==============================] - 2s 48ms/step - loss: 0.2571 - f_score: 0.8742 - precision: 0.8976 - recall: 0.9019
train error =  0.038176119327545166
test error =  0.12582576274871826


## AlexNet

In [27]:
train_seq, val_seq, test_seq = get_dataset_sequences('train-256', 16)

In [4]:
def alexnet():
    inputs = layers.Input(shape=(256, 256, 3))
    
    nn = layers.Conv2D(filters=96, kernel_size=11, strides=(4, 4), activation='relu')(inputs)
    nn = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(nn)
    
    nn = layers.Conv2D(filters=256, kernel_size=5, strides=(4, 4), activation='relu', padding='same')(nn)
    nn = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(nn)
    
    nn = layers.Conv2D(filters=384, kernel_size=3, activation='relu', padding='same')(nn)
    nn = layers.Conv2D(filters=384, kernel_size=3, activation='relu', padding='same')(nn)
    nn = layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(nn)
    nn = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(nn)
    
    nn = layers.Flatten()(nn)
    
    nn = layers.Dense(9216, activation='relu')(nn)
    nn = layers.Dense(4096, activation='relu')(nn)
    outputs = layers.Dense(1, activation='sigmoid')(nn)
    
    model = models.Model(inputs, outputs)
    
    train_metrics = [f_score,
                     metrics.Precision(thresholds=0.5),
                     metrics.Recall(thresholds=0.5)]

    train_optimizer = optimizers.Adam(learning_rate=0.0001)
    train_loss = 'binary_crossentropy'

    model.compile(loss=train_loss, optimizer=train_optimizer, metrics=train_metrics)
    return model

In [5]:
model = alexnet()

2022-04-07 10:49:26.809639: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-07 10:49:26.809992: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-07 10:49:26.810287: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-07 10:49:26.810517: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Epoch 1/100


2022-04-07 10:49:28.704984: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8400
2022-04-07 10:49:29.097740: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-04-07 10:49:29.217250: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 466.50MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-04-07 10:49:29.232329: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 358.56MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-04-07 10:49:29.232364: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate

1011/1407 [====================>.........] - ETA: 31s - loss: 0.6741 - f_score: 0.5338 - precision: 0.5834 - recall: 0.6194

2022-04-07 10:50:49.390674: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 431.97MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


1407/1407 [==============================] - ETA: 0s - loss: 0.6529 - f_score: 0.5686 - precision: 0.6107 - recall: 0.6390
Epoch 1: val_loss improved from inf to 0.55941, saving model to dogs_vs_cats_lenet5_weights_1.h5
1407/1407 [==============================] - 116s 81ms/step - loss: 0.6529 - f_score: 0.5686 - precision: 0.6107 - recall: 0.6390 - val_loss: 0.5594 - val_f_score: 0.7275 - val_precision: 0.7617 - val_recall: 0.7161
Epoch 2/100
1407/1407 [==============================] - ETA: 0s - loss: 0.5223 - f_score: 0.7328 - precision: 0.7412 - recall: 0.7569
Epoch 2: val_loss improved from 0.55941 to 0.45884, saving model to dogs_vs_cats_lenet5_weights_2.h5
1407/1407 [==============================] - 113s 80ms/step - loss: 0.5223 - f_score: 0.7328 - precision: 0.7412 - recall: 0.7569 - val_loss: 0.4588 - val_f_score: 0.7789 - val_precision: 0.8256 - val_recall: 0.7539
Epoch 3/100
1407/1407 [==============================] - ETA: 0s - loss: 0.4359 - f_score: 0.7896 - precision: 0

In [ ]:
train_model(model, train_seq, val_seq)

In [6]:
print_errors(model, train_seq, test_seq)

79/79 [==============================] - 2s 23ms/step - loss: 0.4582 - f_score: 0.7696 - precision: 0.7931 - recall: 0.8006
train error =  0.08798092603683472
test error =  0.2304016351699829


In [13]:
def alexnet_v2(hp):
    inputs = layers.Input(shape=(256, 256, 3))
    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_1', min_value=1, max_value=100, step=1),
                       kernel_size=11, strides=(4, 4), activation='relu')(inputs)
    nn = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(nn)
    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_2', min_value=200, max_value=300, step=1),
                       kernel_size=5, strides=(4, 4), activation='relu', padding='same')(nn)
    nn = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(nn)
    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_3', min_value=300, max_value=400, step=1),
                       kernel_size=3, activation='relu', padding='same')(nn)
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_4', min_value=300, max_value=400, step=1),
                       kernel_size=3, activation='relu', padding='same')(nn)
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_5', min_value=200, max_value=300, step=1),
                       kernel_size=3, activation='relu', padding='same')(nn)
    nn = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(nn)
    
    nn = layers.Flatten()(nn)
    
    nn = layers.Dense(hp.Int(name='dense_1', min_value=2000, max_value=10000, step=50), 
                      activation='relu')(nn)
    nn = layers.Dense(hp.Int(name='dense_2', min_value=2000, max_value=5000, step=50), activation='relu')(nn)
    outputs = layers.Dense(1, activation='sigmoid')(nn)
    
    model = models.Model(inputs, outputs)
    
    train_metrics = [f_score,
                     metrics.Precision(thresholds=0.5),
                     metrics.Recall(thresholds=0.5)]

    train_optimizer = optimizers.Adam(
        learning_rate=hp.Float(name='learning_rate', min_value=10e-6, max_value=10e-4, sampling='log'))
    train_loss = 'binary_crossentropy'

    model.compile(loss=train_loss, optimizer=train_optimizer, metrics=train_metrics)
    return model

In [14]:
tuner = tune_model(alexnet_v2, train_seq, val_seq)

Trial 112 Complete [00h 04m 36s]
val_loss: 0.5043529272079468

Best val_loss So Far: 0.3775126039981842
Total elapsed time: 12h 12m 12s

Search: Running Trial #113

Value             |Best Value So Far |Hyperparameter
43                |81                |filters_conv_1
292               |267               |filters_conv_2
350               |357               |filters_conv_3
366               |349               |filters_conv_4
280               |262               |filters_conv_5
4650              |5000              |dense_1
2350              |4950              |dense_2
0.00011628        |5.5126e-05        |learning_rate
30                |30                |tuner/epochs
6                 |6                 |tuner/initial_epoch
1                 |2                 |tuner/bracket
1                 |2                 |tuner/round
0106              |0034              |tuner/trial_id

Epoch 7/30
1407/1407 [==============================] - 69s 48ms/step - loss: 0.6465 - f_score: 0.5905 - pre

KeyboardInterrupt: 

In [ ]:
alexnet_v2_best_model = tuner.get_best_models()[0]
print_errors(alexnet_v2_best_model, train_seq, test_seq)

In [11]:
alexnet_v2_best_model_trained = train_model(alexnet_v2_best_model, train_seq, val_seq)
print_errors(alexnet_v2_best_model_trained, train_seq, test_seq)

Epoch 1/100
1407/1407 [==============================] - ETA: 0s - loss: 0.0948 - f_score: 0.9602 - precision: 0.9640 - recall: 0.9618
Epoch 1: val_loss improved from inf to 0.95002, saving model to dogs_vs_cats_lenet5_weights_1.h5
1407/1407 [==============================] - 84s 60ms/step - loss: 0.0948 - f_score: 0.9602 - precision: 0.9640 - recall: 0.9618 - val_loss: 0.9500 - val_f_score: 0.7298 - val_precision: 0.8947 - val_recall: 0.6435
Epoch 2/100
1407/1407 [==============================] - ETA: 0s - loss: 0.0813 - f_score: 0.9676 - precision: 0.9699 - recall: 0.9695
Epoch 2: val_loss improved from 0.95002 to 0.61500, saving model to dogs_vs_cats_lenet5_weights_2.h5
1407/1407 [==============================] - 84s 60ms/step - loss: 0.0813 - f_score: 0.9676 - precision: 0.9699 - recall: 0.9695 - val_loss: 0.6150 - val_f_score: 0.8223 - val_precision: 0.7911 - val_recall: 0.8722
Epoch 3/100
1407/1407 [==============================] - ETA: 0s - loss: 0.0715 - f_score: 0.9712 - pr

## Lenet5 v6

In [28]:
def lenet5_v6(hp):
    
    inputs = layers.Input(shape=(256, 256, 3))
    nn = layers.Rescaling(1.0 / 255.0)(inputs)

    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_1', min_value=1, max_value=20, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(inputs)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)

    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_2', min_value=20, max_value=40, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)

    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_3', min_value=40, max_value=60, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu', padding='same')(nn)
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_4', min_value=40, max_value=60, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)
    
    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_5', min_value=60, max_value=80, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu', padding='same')(nn)
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_6', min_value=60, max_value=80, step=1),
                       kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)
    
    
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_7', min_value=80, max_value=100, step=1),
                        kernel_size=5, strides=(1, 1), activation='relu', padding='same')(nn)
    nn = layers.Conv2D(filters=hp.Int(name='filters_conv_7', min_value=80, max_value=100, step=1),
                        kernel_size=5, strides=(1, 1), activation='relu')(nn)
    nn = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(nn)

    
    nn = layers.Flatten()(nn)

    nn = layers.Dense(hp.Int(name='units_dense_1', min_value=300, max_value=500, step=1), activation='relu')(nn)
    nn = layers.Dense(hp.Int(name='units_dense_2', min_value=80, max_value=120, step=1), activation='relu')(nn)
    nn = layers.Dense(hp.Int(name='units_dense_3', min_value=40, max_value=80, step=1), activation='relu')(nn)
    outputs = layers.Dense(1, activation='sigmoid')(nn)

    model = models.Model(inputs, outputs)
    
    train_metrics = [f_score,
                     metrics.Precision(thresholds=0.5),
                     metrics.Recall(thresholds=0.5)]

    train_optimizer = optimizers.Adam(
        learning_rate=hp.Float(name='learning_rate', min_value=10e-6, max_value=10e-4, sampling='log'))
    train_loss = 'binary_crossentropy'

    model.compile(loss=train_loss, optimizer=train_optimizer, metrics=train_metrics)
    return model

In [20]:
tuner = tune_model(lenet5_v6, train_seq, val_seq)

Trial 104 Complete [00h 19m 24s]
val_loss: 0.27580711245536804

Best val_loss So Far: 0.16066938638687134
Total elapsed time: 22h 44m 35s

Search: Running Trial #105

Value             |Best Value So Far |Hyperparameter
18                |8                 |filters_conv_1
31                |36                |filters_conv_2
48                |49                |filters_conv_3
51                |46                |filters_conv_4
68                |72                |filters_conv_5
63                |78                |filters_conv_6
85                |81                |filters_conv_7
339               |399               |units_dense_1
80                |88                |units_dense_2
47                |58                |units_dense_3
5.7848e-05        |0.00017896        |learning_rate
6                 |30                |tuner/epochs
0                 |0                 |tuner/initial_epoch
1                 |0                 |tuner/bracket
0                 |0                 |tu

KeyboardInterrupt: 

In [29]:
tuner = kt.Hyperband(
        lenet5_v6,
        objective="val_loss",
        max_epochs=30,
        hyperband_iterations=3,
        factor=5,
        directory='tuner',
        project_name='lenet5_v6',
    )
lenet_v6_best_model = tuner.get_best_models()[0]
print_errors(lenet_v6_best_model, train_seq, test_seq)

INFO:tensorflow:Reloading Oracle from existing project tuner/lenet5_v6/oracle.json
INFO:tensorflow:Reloading Tuner from tuner/lenet5_v6/tuner0.json
79/79 [==============================] - 3s 39ms/step - loss: 0.2270 - f_score: 0.9007 - precision: 0.9346 - recall: 0.9051
train error =  0.01674330234527588
test error =  0.0992971658706665


In [ ]:
lenet_v6_best_model_trained = train_model(lenet_v6_best_model, train_seq, val_seq)
print_errors(lenet_v6_best_model_trained, train_seq, test_seq)

Epoch 1/100
1407/1407 [==============================] - ETA: 0s - loss: 0.0550 - f_score: 0.9778 - precision: 0.9771 - recall: 0.9808
Epoch 1: val_loss improved from inf to 0.20775, saving model to dogs_vs_cats_lenet5_weights_1.h5
1407/1407 [==============================] - 149s 105ms/step - loss: 0.0550 - f_score: 0.9778 - precision: 0.9771 - recall: 0.9808 - val_loss: 0.2077 - val_f_score: 0.9237 - val_precision: 0.9374 - val_recall: 0.9211 - lr: 1.7896e-04
Epoch 2/100
1407/1407 [==============================] - ETA: 0s - loss: 0.0587 - f_score: 0.9777 - precision: 0.9770 - recall: 0.9816
Epoch 2: val_loss improved from 0.20775 to 0.18937, saving model to dogs_vs_cats_lenet5_weights_2.h5
1407/1407 [==============================] - 148s 105ms/step - loss: 0.0587 - f_score: 0.9777 - precision: 0.9770 - recall: 0.9816 - val_loss: 0.1894 - val_f_score: 0.9273 - val_precision: 0.9554 - val_recall: 0.9132 - lr: 1.7896e-04
Epoch 3/100
1407/1407 [==============================] - ETA: 0s

Epoch 20/100
1407/1407 [==============================] - ETA: 0s - loss: 7.8167e-06 - f_score: 1.0000 - precision: 1.0000 - recall: 1.0000
Epoch 20: val_loss did not improve from 0.18937
1407/1407 [==============================] - 148s 105ms/step - loss: 7.8167e-06 - f_score: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.6362 - val_f_score: 0.9350 - val_precision: 0.9415 - val_recall: 0.9401 - lr: 1.4317e-06
Epoch 21/100
1407/1407 [==============================] - ETA: 0s - loss: 5.2678e-06 - f_score: 1.0000 - precision: 1.0000 - recall: 1.0000
Epoch 21: val_loss did not improve from 0.18937
1407/1407 [==============================] - 148s 105ms/step - loss: 5.2678e-06 - f_score: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.6676 - val_f_score: 0.9350 - val_precision: 0.9415 - val_recall: 0.9401 - lr: 1.4317e-06
Epoch 22/100
1407/1407 [==============================] - ETA: 0s - loss: 3.3052e-06 - f_score: 1.0000 - precision: 1.0000 - recall: 1.0000
Epoch 22